Downloading the Requirements

In [2]:
# Install required package first, then import and setup

# Install openai
!pip install openai

# Now import after installation
import os
import json
from openai import OpenAI
from typing import List, Dict, Any, Optional
import math

# Set Groq API key (provided by user)
os.environ["GROQ_API_KEY"] = "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"

# Verify API key
if not os.getenv("GROQ_API_KEY"):
    raise ValueError("GROQ_API_KEY is not set.")

# Initialize OpenAI client for Groq API
client = OpenAI(
    api_key=os.getenv("GROQ_API_KEY"),
    base_url="https://api.groq.com/openai/v1",
)
print("Setup complete. Groq API client initialized.")

Setup complete. Groq API client initialized.


ConversationManager Class

In [7]:
# Define the ConversationManager class with all necessary methods for Task 1

from typing import List, Dict, Any, Optional  # Ensure typing imports are available

class ConversationManager:
    def __init__(self, model: str = "llama3-8b-8192", summarize_every: int = 3):
        self.history: List[Dict[str, str]] = []
        self.model = model
        self.summarize_every = summarize_every
        self.turn_count = 0

    def add_turn(self, role: str, content: str) -> None:
        """Add a user or assistant message to history."""
        self.history.append({"role": role, "content": content})
        self.turn_count += 1
        if self.turn_count % self.summarize_every == 0:
            self._summarize_history()

    def truncate_by_turns(self, max_turns: int) -> None:
        """Truncate history to last max_turns messages."""
        if len(self.history) > max_turns:
            self.history = self.history[-max_turns:]

    def truncate_by_length(self, max_length: int, unit: str = "chars") -> None:
        """Truncate history to total max_length chars or words."""
        total = 0
        if unit == "chars":
            total = sum(len(msg["content"]) for msg in self.history)
        elif unit == "words":
            total = sum(len(msg["content"].split()) for msg in self.history)

        if total > max_length:
            while total > max_length and len(self.history) > 1:
                removed = self.history.pop(0)
                if unit == "chars":
                    total -= len(removed["content"])
                elif unit == "words":
                    total -= len(removed["content"].split())

    def _summarize_history(self) -> None:
        """Summarize history using Groq API and replace with summary."""
        if len(self.history) < 2:
            return

        history_text = "\n".join([f"{msg['role'].title()}: {msg['content']}" for msg in self.history])
        prompt = f"Summarize the following conversation concisely, keeping key points and context:\n\n{history_text}"

        response = client.chat.completions.create(
            model=self.model,
            messages=[{"role": "user", "content": prompt}],
            max_tokens=500,
            temperature=0.3,
        )
        summary = response.choices[0].message.content.strip()

        self.history = [
            {"role": "system", "content": f"Summary of previous conversation: {summary}"},
            self.history[-1]
        ]
        print(f"Summarization triggered! New history length: {len(self.history)}")

    def get_history(self) -> List[Dict[str, str]]:
        """Return current history."""
        return self.history[:]

    def chat(self, user_input: str) -> str:
        """Simulate a chat turn: add user input, generate response, handle summarization."""
        self.add_turn("user", user_input)

        messages = self.get_history()
        response = client.chat.completions.create(
            model=self.model,
            messages=messages,
            max_tokens=200,
            temperature=0.7,
        )
        assistant_reply = response.choices[0].message.content.strip()

        self.add_turn("assistant", assistant_reply)
        return assistant_reply

print("ConversationManager class defined.")

ConversationManager class defined.


Full Conversation Demo

In [10]:
# Demonstrate conversation with no truncation, showing all turns
# Updated model to 'llama-3.1-8b-instant' as 'llama3-8b-8192' is deprecated

print("=== Task 1: Full Conversation Demo ===")

# Create manager with updated model (deprecated model replaced)
manager = ConversationManager(model="llama-3.1-8b-instant", summarize_every=3)

# Sample conversation inputs
sample_inputs = [
    "Hello, what's the weather like today?",
    "Tell me a joke.",
    "What's the capital of France?",
    "Explain quantum physics simply.",
    "Recommend a book on AI.",
    "What's your favorite color?",
]

# Run conversation
for i, inp in enumerate(sample_inputs, 1):
    reply = manager.chat(inp)
    print(f"Turn {i}: User: {inp}\nAssistant: {reply}\n")

print("Final history:")
print(json.dumps(manager.get_history(), indent=2))

=== Task 1: Full Conversation Demo ===
Turn 1: User: Hello, what's the weather like today?
Assistant: However, I'm a large language model, I don't have real-time access to current weather conditions. I can suggest some options to help you find out the weather today:

1. Check online weather websites: You can visit websites like AccuWeather, Weather.com, or the National Weather Service (NWS) for the current weather conditions in your area.
2. Use a mobile app: Many mobile apps such as Dark Sky, Weather Underground, or The Weather Channel provide real-time weather updates.
3. Check your phone's built-in weather app: If your phone has a built-in weather app, you can check the current weather conditions.
4. Ask a voice assistant: You can ask a voice assistant like Siri, Google Assistant, or Alexa to provide you with the current weather conditions.

Please let me know if there's anything else I can help you with.

Summarization triggered! New history length: 2
Turn 2: User: Tell me a joke.


Task 1 - Truncation by Turns Demo

In [12]:
# Demonstrate truncation by keeping last 4 turns
# Note: sample_inputs must be defined from Cell 3 or re-defined here

print("=== Task 1: Truncation by Turns (Last 4) ===")

# Re-define sample_inputs if not available from previous cell
sample_inputs = [
    "Hello, what's the weather like today?",
    "Tell me a joke.",
    "What's the capital of France?",
    "Explain quantum physics simply.",
    "Recommend a book on AI.",
    "What's your favorite color?",
]

# Create manager with updated model and disable periodic summarization for clarity
manager_trunc_turns = ConversationManager(model="llama-3.1-8b-instant", summarize_every=5)

# Run full conversation
print("Running full conversation...")
for inp in sample_inputs:
    reply = manager_trunc_turns.chat(inp)
    print(f"User: {inp} -> Assistant: {reply[:50]}...")  # Show truncated reply for brevity

print("\nBefore truncation - Full history length:", len(manager_trunc_turns.get_history()))
print("Before truncation - History:")
print(json.dumps(manager_trunc_turns.get_history(), indent=2)[:500] + "..." if len(json.dumps(manager_trunc_turns.get_history(), indent=2)) > 500 else json.dumps(manager_trunc_turns.get_history(), indent=2))

# Apply truncation to keep only last 4 turns
manager_trunc_turns.truncate_by_turns(4)

print("\nAfter truncation (last 4 turns):")
print("History length:", len(manager_trunc_turns.get_history()))
print(json.dumps(manager_trunc_turns.get_history(), indent=2))

=== Task 1: Truncation by Turns (Last 4) ===
Running full conversation...
User: Hello, what's the weather like today? -> Assistant: However, I'm a large language model, I don't have ...
User: Tell me a joke. -> Assistant: Here's one:

What do you call a fake noodle?

An i...
Summarization triggered! New history length: 2
User: What's the capital of France? -> Assistant: The capital of France is Paris....
User: Explain quantum physics simply. -> Assistant: Quantum physics is a branch of physics that studie...
Summarization triggered! New history length: 2
User: Recommend a book on AI. -> Assistant: Here are a few book recommendations on AI:

1. **"...
User: What's your favorite color? -> Assistant: I don't have a favorite color. I'm a machine learn...

Before truncation - Full history length: 4
Before truncation - History:
[
  {
    "role": "system",
    "content": "Summary of previous conversation: Here's a concise summary of the conversation:\n\n- The user initially asked for the capi

Task 1 - Truncation by Length Demo

In [14]:
# Demonstrate truncation by character limit (500 chars)
# Note: sample_inputs must be defined from previous cells or re-defined here

print("=== Task 1: Truncation by Length (500 chars) ===")

# Re-define sample_inputs if not available from previous cell
sample_inputs = [
    "Hello, what's the weather like today?",
    "Tell me a joke.",
    "What's the capital of France?",
    "Explain quantum physics simply.",
    "Recommend a book on AI.",
    "What's your favorite color?",
]

# Create manager with updated model and disable periodic summarization for clarity
manager_trunc_len = ConversationManager(model="llama-3.1-8b-instant", summarize_every=5)

# Run full conversation
print("Running full conversation...")
for inp in sample_inputs:
    reply = manager_trunc_len.chat(inp)
    print(f"User: {inp} -> Assistant: {reply[:50]}...")  # Show truncated reply for brevity

# Calculate total characters before truncation
full_history = manager_trunc_len.get_history()
total_chars_before = sum(len(msg["content"]) for msg in full_history)
print(f"\nBefore truncation - Full history length: {len(full_history)} messages")
print(f"Before truncation - Total characters: {total_chars_before}")
print("Before truncation - History preview:")
print(json.dumps(full_history, indent=2)[:500] + "..." if len(json.dumps(full_history, indent=2)) > 500 else json.dumps(full_history, indent=2))

# Apply truncation by character limit
manager_trunc_len.truncate_by_length(500, "chars")
truncated_history = manager_trunc_len.get_history()
total_chars_after = sum(len(msg["content"]) for msg in truncated_history)

print(f"\nAfter truncation (max 500 chars):")
print(f"History length: {len(truncated_history)} messages")
print(f"Total characters: {total_chars_after}")
print("Truncated history:")
print(json.dumps(truncated_history, indent=2))

=== Task 1: Truncation by Length (500 chars) ===
Running full conversation...
User: Hello, what's the weather like today? -> Assistant: However, I'm a large language model, I don't have ...
User: Tell me a joke. -> Assistant: Why don't scientists trust atoms?

Because they ma...
Summarization triggered! New history length: 2
User: What's the capital of France? -> Assistant: The capital of France is Paris....
User: Explain quantum physics simply. -> Assistant: Quantum physics is a branch of physics that studie...
Summarization triggered! New history length: 2
User: Recommend a book on AI. -> Assistant: Here are some popular and highly-recommended books...
User: What's your favorite color? -> Assistant: I don't have a personal preference, including favo...

Before truncation - Full history length: 4 messages
Before truncation - Total characters: 1816
Before truncation - History preview:
[
  {
    "role": "system",
    "content": "Summary of previous conversation: Here's a concise summary

Task 1 - Periodic Summarization Demo

In [16]:
# Demonstrate summarization every 3 turns
# Note: sample_inputs must be defined from previous cells or re-defined here

print("=== Task 1: Periodic Summarization (Every 3 Turns) ===")

# Re-define sample_inputs if not available from previous cell
sample_inputs = [
    "Hello, what's the weather like today?",
    "Tell me a joke.",
    "What's the capital of France?",
    "Explain quantum physics simply.",
    "Recommend a book on AI.",
    "What's your favorite color?",
]

# Create manager with updated model and summarization every 3 turns
manager_periodic = ConversationManager(model="llama-3.1-8b-instant", summarize_every=3)

# Run conversation for first 6 inputs (to trigger 2 summaries)
for i, inp in enumerate(sample_inputs[:6], 1):
    reply = manager_periodic.chat(inp)
    print(f"Turn {i}: User: {inp}\nAssistant: {reply}\n")
    if i % 3 == 0:
        print(f"--- Summary after turn {i} ---")
        print("Current history after summary:")
        print(json.dumps(manager_periodic.get_history(), indent=2))

print("Final history after all summaries:")
print(json.dumps(manager_periodic.get_history(), indent=2))

=== Task 1: Periodic Summarization (Every 3 Turns) ===
Turn 1: User: Hello, what's the weather like today?
Assistant: However, I'm a large language model, I don't have real-time access to your current location's weather. But I can suggest a few options to help you find out the weather today:

1. **Check online weather websites**: You can visit websites like AccuWeather, Weather.com, or the National Weather Service (NWS) to get the current weather conditions and forecast for your area.
2. **Use a mobile app**: You can download a weather app on your smartphone, such as Dark Sky or Weather Underground, which can provide you with real-time weather updates and forecasts.
3. **Ask a voice assistant**: If you have a smart speaker or virtual assistant, such as Siri, Google Assistant, or Alexa, you can ask it to tell you the weather in your area.

If you provide me with your location or city, I can give you a general idea of the weather based on historical data.

Summarization triggered! New hi

 Task 2 - JSON Schema and Extraction Functions

In [18]:
# Define JSON schema and functions for structured extraction
# Updated model to supported 'llama-3.1-8b-instant' to avoid deprecation errors

# JSON Schema for personal info extraction
extraction_schema = {
    "type": "object",
    "properties": {
        "name": {"type": "string", "description": "User's full name"},
        "email": {"type": "string", "description": "User's email address"},
        "phone": {"type": "string", "description": "User's phone number"},
        "location": {"type": "string", "description": "User's location (city/country)"},
        "age": {"type": "integer", "description": "User's age"}
    },
    "required": ["name", "email", "phone", "location", "age"]
}

# Tool for function calling
extraction_tool = {
    "type": "function",
    "function": {
        "name": "extract_personal_info",
        "description": "Extract personal information from the chat.",
        "parameters": extraction_schema
    }
}

def extract_info_from_chat(chat_text: str, model: str = "llama-3.1-8b-instant") -> Optional[Dict[str, Any]]:
    """Extract structured info using Groq API function calling."""
    messages = [
        {"role": "system", "content": "Extract personal details from the conversation using the provided tool."},
        {"role": "user", "content": chat_text}
    ]

    response = client.chat.completions.create(
        model=model,
        messages=messages,
        tools=[extraction_tool],
        tool_choice="auto",
        temperature=0.1,
    )

    if response.choices[0].message.tool_calls:
        tool_call = response.choices[0].message.tool_calls[0]
        if tool_call.function.name == "extract_personal_info":
            return json.loads(tool_call.function.arguments)
    return None

def validate_extraction(extracted: Dict[str, Any]) -> bool:
    """Validate extracted data against schema."""
    required = ["name", "email", "phone", "location", "age"]
    if not all(key in extracted for key in required):
        return False
    if not isinstance(extracted["name"], str) or not isinstance(extracted["email"], str) or not isinstance(extracted["phone"], str):
        return False
    if not isinstance(extracted["location"], str) or not isinstance(extracted["age"], int):
        return False
    if "@" not in extracted["email"] or not extracted["phone"].startswith(("+", "1")):
        return False
    if extracted["age"] < 0 or extracted["age"] > 150:
        return False
    return True

print("Task 2 functions and schema defined.")

Task 2 functions and schema defined.


Task 2 - Extraction and Validation Demo

In [19]:
# Demonstrate extraction and validation on 3 sample chats

print("=== Task 2: Extraction and Validation ===")
sample_chats = [
    "Hi, I'm John Doe, 28 years old, living in New York, USA. My email is john.doe@example.com and phone is +1-555-1234.",
    "Hello! My name is Alice Smith, age 35, from London, UK. Contact me at alice.smith@ukmail.com or +44-20-1234-5678.",
    "Hey, Sarah Johnson here, 42, in Tokyo, Japan. Email: sarah.j@asia.net, phone: +81-3-1234-5678.",
]

for i, chat in enumerate(sample_chats, 1):
    print(f"\n--- Sample Chat {i} ---")
    print(f"Chat: {chat}")

    extracted = extract_info_from_chat(chat)
    if extracted:
        print("Extracted:", json.dumps(extracted, indent=2))
        is_valid = validate_extraction(extracted)
        print(f"Validation: {'PASS' if is_valid else 'FAIL'}")
    else:
        print("Extraction failed: No tool call.")

=== Task 2: Extraction and Validation ===

--- Sample Chat 1 ---
Chat: Hi, I'm John Doe, 28 years old, living in New York, USA. My email is john.doe@example.com and phone is +1-555-1234.
Extracted: {
  "age": 28,
  "email": "john.doe@example.com",
  "location": "New York, USA",
  "name": "John Doe",
  "phone": "+1-555-1234"
}
Validation: PASS

--- Sample Chat 2 ---
Chat: Hello! My name is Alice Smith, age 35, from London, UK. Contact me at alice.smith@ukmail.com or +44-20-1234-5678.
Extracted: {
  "age": 35,
  "email": "alice.smith@ukmail.com",
  "location": "London, UK",
  "name": "Alice Smith",
  "phone": "+44-20-1234-5678"
}
Validation: PASS

--- Sample Chat 3 ---
Chat: Hey, Sarah Johnson here, 42, in Tokyo, Japan. Email: sarah.j@asia.net, phone: +81-3-1234-5678.
Extracted: {
  "age": 42,
  "email": "sarah.j@asia.net",
  "location": "Tokyo, Japan",
  "name": "Sarah Johnson",
  "phone": "+81-3-1234-5678"
}
Validation: PASS
